# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [4]:
# System imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
import panel as pn
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact
from panel import widgets
from string import digits
from numpy import array
from dotenv import load_dotenv
import csv
import json
import numpy as np
import dateparser
import panel as pn
import matplotlib.ticker as tkr

plt.style.use('seaborn-white')

pn.extension('plotly')

pn.extension()

%matplotlib inline


In [6]:
# Read the Mapbox API key
mapbox_token = os.getenv("MAPBOX_API_KEY")
load_dotenv()

True

In [7]:
# Local imports
import sys
sys.path.append("../lib2")

from Constants import Constants
from ProcessingTools import DateProcessingTools


# Import Data

In [8]:
# Construct the tools
debug_level = 0
constants = Constants()
tool_data_processing = DateProcessingTools(debug_level)


In [9]:
# Import all preprocessed data
atlas           = pd.read_pickle(constants.PREPROCESSED_ATLAS_FILE_PATH)
forecast        = pd.read_pickle(constants.PREPROCESSED_FORECAST_DATA_FILE_PATH)
revenue2020     = pd.read_pickle(constants.PREPROCESSED_REVENUE2020_FILE_PATH)
revenue2020A    = pd.read_pickle(constants.PREPROCESSED_REVENUE2020A_FILE_PATH)


In [10]:
atlas = atlas.sort_values(by='Invoice Date')

In [11]:
atlas.reset_index(inplace=True)
atlas

,Customers,Number of Users,Invoice Date,Invoice #,Year,Month,Invoice Amount,Subscription,2021 Prediction,Customers Status,Account Code,Address,State,Lat,Long,Service Start,Service End,Subscription Duration
0,Louisiana Board of Regents,7050.0,2015-03-20 00:00:00-04:00,ATLAS 315,2015,March,72000.0,1 Year,NaN,Lost,4700-0-00-00000-18-0000,"1201 N 3rd St #6, Baton Rouge, LA 70802",LA,30.460190,-91.188530,2015-03-18 00:00:00-04:00,2016-06-30 00:00:00-04:00,365 days
1,Emporia State University,250.0,2015-05-28 00:00:00-04:00,AJ501,2015,May,3500.0,1 Year,NaN,Active,4700-0-00-00000-16-0000,"1 Kellogg Cir, Emporia, KS 66801",KS,38.392609,-96.181396,2015-06-01 00:00:00-04:00,2016-06-30 00:00:00-04:00,365 days
2,Morehead State University,100.0,2015-06-23 00:00:00-04:00,AJ502,2015,June,3500.0,1 Year,NaN,Active,4700-0-00-00000-17-0000,"150 University Blvd, Morehead, KY 40351",KY,38.184921,-83.434441,2015-06-10 00:00:00-04:00,2016-06-30 00:00:00-04:00,365 days
3,State University of NY Potsdam,500.0,2015-06-26 00:00:00-04:00,AJ503,2015,June,6500.0,1 Year,NaN,Lost,4700-0-00-00000-32-0000,"44 Pierrepont Ave, Potsdam, NY 13676",NY,44.663780,-74.978409,2015-06-01 00:00:00-04:00,2016-06-30 00:00:00-04:00,365 days
4,ST. Mary's College of Maryland,15.0,2015-10-07 00:00:00-04:00,AJ504,2015,October,750.0,1 Year,NaN,Lost,4700-0-00-00000-20-0000,"47645 College Dr, St Marys City, MD 20686",MD,38.190601,-76.426300,2015-10-05 00:00:00-04:00,2016-09-30 00:00:00-04:00,365 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,Modesto Junior College,200.0,2020-12-08 00:00:00-05:00,NB120920,2020,December,1750.0,6 Months,"$1,750.00",New,4700-0-00-00000-00-0000,"435 College Ave, Modesto, CA 95350",CA,37.651310,-121.009726,2021-03-01 00:00:00-05:00,2021-08-31 00:00:00-04:00,180 days
696,University of North Carolina Wilmington,200.0,2020-12-08 00:00:00-05:00,NB12092B,2020,December,2000.0,1 Year,"$2,000.00",Active,4700-0-00-00000-00-0000,"601 S College Rd, Wilmington, NC 28403",NC,34.223991,-77.867764,2020-08-31 00:00:00-04:00,2021-08-31 00:00:00-04:00,365 days
697,Brock University,350.0,2020-12-08 00:00:00-05:00,NB120820,2020,December,2500.0,6 Months,"$2,500.00",New,4700-0-00-00000-00-0000,"1812 Sir Isaac Brock Way, St. Catharines, Onta...",Canada,43.118799,-79.238508,2020-12-09 00:00:00-05:00,2021-05-31 00:00:00-04:00,180 days
698,Holyoke Community College,50.0,2020-12-08 00:00:00-05:00,NB120920A,2020,December,1000.0,1 Year,$500.00,New,4700-0-00-00000-00-0000,"303 Homestead Avenue, Holyoke, MA 01040",MA,42.193910,-72.647450,2021-01-01 00:00:00-05:00,2021-12-31 00:00:00-05:00,365 days


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [4]:
# Define Panel Visualization Functions
def Housing_Units_Per_Year():
    """Housing Units Per Year."""
    Mean_Housing_Units = SFO_Data.groupby(["year"])["housing_units"].mean()
    SFO_Grouped_MIN = SFO_Data.groupby(["year"])["housing_units"].min().min()
    SFO_Grouped_MAX = SFO_Data.groupby(["year"])["housing_units"].max().max()
    SFO_Grouped_STD = SFO_Data["housing_units"].std()
    #TODO fix years...
    First_Year = Mean_Housing_Units.index[0]
    Last_Year = Mean_Housing_Units.index[-1]
    Fig_Housing_Units = px.bar(Mean_Housing_Units,
                               title= "Housing Units from 2010 to 2016 in San Francisco")
    return Fig_Housing_Units

def Average_Gross_Rent():
    """Average Gross Rent in San Francisco Per Year."""
    SFO_Grouped_by_Year = SFO_Data.groupby(["year"])
    Mean_Gross_Rent = SFO_Grouped_by_Year["gross_rent"].mean()
    Mean_Gross_Rent_Plot = px.line(Mean_Gross_Rent,
                                   title ="Average Gross Rent in San Francisco")
    return Mean_Gross_Rent_Plot

def Average_Sales_Price():
    """Average Sales Price Per Year."""
    SFO_Grouped_by_Year = SFO_Data.groupby(["year"])
    Mean_Price_per_SQFT = SFO_Grouped_by_Year["sale_price_sqr_foot"].mean()
    Mean_Price_per_SQFT_Plot = px.line(Mean_Price_per_SQFT,
                                       title = "Average Sales Price per Year in San Francisco")
    return Mean_Price_per_SQFT_Plot

def Average_Price_by_Neighborhood():
    """Average Prices by Neighborhood."""
    SFO_Neighborhood = pd.read_csv("Data/sfo_neighborhoods_census_data.csv")
    SFO_Grouped_by_Year_and_NBHD = SFO_Neighborhood.groupby(["year", "neighborhood"])
    SFO_Grouped_by_Year_and_NBHD.head()
    Avg_Price_Mean = SFO_Grouped_by_Year_and_NBHD.mean()
    SFO_Grouped_Year_and_NBHD_Plot = Avg_Price_Mean.hvplot.line(
        x='year',
        y='sale_price_sqr_foot',
        xlabel='Year',
        ylabel='Price per Sqr Foot',
        groupby='neighborhood',
        width=600
        ).opts(yformatter='%0f')
    return SFO_Grouped_Year_and_NBHD_Plot

def TOP_Expensive_Neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    Groupby_Neighborhood = SFO_Data.groupby(["neighborhood"]).mean()
    TOP_10_Expensive = Groupby_Neighborhood.sort_values(by="sale_price_sqr_foot", ascending=False).head(10)
    TOP_10_Expensive = TOP_10_Expensive.reset_index()
    TOP_10_Expensive_Plot = TOP_10_Expensive.hvplot.bar(
        x="neighborhood",
        y="sale_price_sqr_foot",
        xlabel = "Neighborhoods",
        ylabel = "Average Sale Price Per Square Foot",
        title="Top 10 Most Expensive Neighborhoods in SFO",
        height=400,
        rot=90)
    return TOP_10_Expensive_Plot

def Parallel_Coordinates():
    """Parallel Coordinates Plot."""
    Groupby_Neighborhood = SFO_Data.groupby(["neighborhood"]).mean()
    TOP_10_Expensive_1 = Groupby_Neighborhood.sort_values(by="sale_price_sqr_foot", ascending=False).head(10)
    TOP_10_Expensive_1 = TOP_10_Expensive_1.reset_index()
    TOP_10_Expensive_Parallel = px.parallel_coordinates(TOP_10_Expensive_1, color='sale_price_sqr_foot')
    return TOP_10_Expensive_Parallel

def Parallel_Categories():
    """Parallel Categories Plot."""
    Groupby_Neighborhood = SFO_Data.groupby(["neighborhood"]).mean()
    TOP_10_Expensive_2 = Groupby_Neighborhood.sort_values(by="sale_price_sqr_foot", ascending=False).head(10)
    TOP_10_Expensive_2 = TOP_10_Expensive_2.reset_index()
    TOP_10_Expensive_Categories =  px.parallel_categories(
            TOP_10_Expensive_2,
            dimensions=["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"],
            color="sale_price_sqr_foot",
            color_continuous_scale=px.colors.sequential.Inferno,
        )
    return TOP_10_Expensive_Categories

def Neighborhood_Map():
    """Neighborhood Map"""
    Mean_Data = SFO_Data.groupby(["neighborhood"]).mean()
    Mean_Data.reset_index(inplace=True)
    Combined_Data = pd.concat([DF_Neighborhood_Locations, Mean_Data], axis='columns', join='inner')
    Combined_Data = Combined_Data.iloc[:, 1:]
    px.set_mapbox_access_token(mapbox_token)
    Neighborhood_Map_Plot = px.scatter_mapbox(
            Combined_Data,
            lat="Lat",
            lon="Lon",
            size="gross_rent",
            color="neighborhood"
        )
    return Neighborhood_Map_Plot

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [5]:
x```

Tabs
    [0] Column
        [0] Plotly(Figure)
    [1] Column
        [0] Plotly(Figure)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [2] Column
        [0] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [1] HoloViews(Bars)
    [3] Column
        [0] Plotly(Figure)
        [1] Plotly(Figure)

## Serve the Panel Dashboard
